# Credit Risk Resampling Techniques

In [75]:
import warnings
warnings.filterwarnings('ignore')

In [76]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split #to for the train_test_split function

# Read the CSV into DataFrame

In [77]:
# Load the data
#using pandas to read in csv file
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.tail() #shows there are over 70,000 rows

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
77531,19100.0,11.261,own,86600,0.653580,12,2,56600,high_risk
77532,17700.0,10.662,mortgage,80900,0.629172,11,2,50900,high_risk
77533,17600.0,10.595,rent,80300,0.626401,11,2,50300,high_risk
77534,16300.0,10.068,mortgage,75300,0.601594,10,2,45300,high_risk
77535,15600.0,9.742,mortgage,72300,0.585062,9,2,42300,high_risk


In [78]:
df= pd.get_dummies(df, columns = ['homeowner'])
#wouldn't let me scale without having all columns in rows display floats/integers
#.get_dummies() converts categorical variables into dummy/indicator variables.

# Split the Data into Training and Testing

In [79]:
# Create our features
X = df.drop(columns="loan_status") #x (features) is all columns without loan_status / x is what we will use to predict

# Create our target
y = df["loan_status"] #y equals this specific column (target)/ what we want to predict
#dealing with 2 classes, high_risk and low_risk

In [84]:
X.shape #data set has 10 columns (doesn't include 'loan status')

(77536, 10)

In [85]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [86]:
# Check the balance of our target values
y.value_counts()
#low_risk is marjoity
#high_risk is minority

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [87]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, #column created above
                                                    y, #column created above
                                                    random_state=1, 
                                                    stratify=y)

#using this data to train and test to see if predictions of actual data is accurate (if so, we can predict new data)

#70% of the data is going to training, 30% going to testing

X_train.shape #as you can see, 58152 out of 77536 rows is around 75% of data

(58152, 10)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [88]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
#Scaling is used so that one column doesn't have too much bias
ds = StandardScaler()

In [89]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

X_scaler = ds.fit(X_train) #using it on X_train (training dataset)


In [90]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [91]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [92]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = classifier.predict(X_test) #predicts y using the X_text (the remaining 25% of data)

balanced_accuracy_score(y_test, y_pred) #compares y_test to y_predictions
#use balanced accuracy score to deal with imbalanced datasets. 

0.9543211898288821

In [93]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [94]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))
#this builds a classification report based on metrics used with imbalanced dataset.

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [95]:
# Resample the training data with the RandomOversampler
#Oversampling adds samples to minority class
from imblearn.over_sampling import RandomOverSampler
#only resample training data
ros = RandomOverSampler(random_state=1)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train) #fits X_train and y_train


# View the count of target classes with Counter
Counter(y_resampled) #now there are the same amount for both

Counter({'low_risk': 56277, 'high_risk': 56277})

In [96]:
# Train the Logistic Regression model using the resampled data
classifier.fit(X_resampled,  y_resampled)

LogisticRegression(random_state=1)

In [97]:
# Calculated the balanced accuracy score
y_pred = classifier.predict(X_test) #predicts y using the X_text (the remaining 25% of data)
balanced_accuracy_score(y_test, y_pred) #compares y_test to y_predictions

0.9947746894823818

In [98]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  106, 18653]])

In [99]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE #allows us to use SMOTE

In [101]:
# Resample the training data with SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [102]:
# Train the Logistic Regression model using the resampled data
classifier.fit(X_resampled,  y_resampled)

LogisticRegression(random_state=1)

In [103]:
# Calculated the balanced accuracy score
y_pred = classifier.predict(X_test) #predicts y using the X_text (the remaining 25% of data)
balanced_accuracy_score(y_test, y_pred) #compares y_test to y_predictions

0.9948279972279972

In [104]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  104, 18655]])

In [105]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))
#slighty better than OverSampling based on confusion matrix

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [107]:
from imblearn.under_sampling import ClusterCentroids #allows us to use ClusterCentroids

In [108]:
# Resample the data using the ClusterCentroids resampler
#decreases size of majority class
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [109]:
# Train the Logistic Regression model using the resampled data
classifier.fit(X_resampled,  y_resampled)

LogisticRegression(random_state=1)

In [110]:
# Calculate the balanced accuracy score
y_pred = classifier.predict(X_test) 
balanced_accuracy_score(y_test, y_pred)

0.9828813049736127

In [111]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  607,    18],
       [  102, 18657]])

In [112]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.97      0.99      0.91      0.98      0.96       625
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18759

avg / total       0.99      0.99      0.97      0.99      0.98      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [114]:
from imblearn.combine import SMOTEENN #allows us to use SMOTEEN

In [115]:
# Resample the training data with SMOTEENN
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 55290, 'low_risk': 55900})

In [116]:
# Train the Logistic Regression model using the resampled data
classifier.fit(X_resampled,  y_resampled)

LogisticRegression(random_state=1)

In [117]:
# Calculate the balanced accuracy score
y_pred = classifier.predict(X_test) 
balanced_accuracy_score(y_test, y_pred)

0.994748035609574

In [118]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  107, 18652]])

In [119]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.


In [ ]:
SMote undersampling did the best for me